In [1]:
#primero importamos todos los paquetes necesarios
import torch #contiene todas las funciones de PyTorch
import torch.nn as nn #contiene la clase padre de todos los modelos (nn.Module)
import torch.nn.functional as F #esencial para la función de activación 
import torchvision #fundamental para la importación de imágenes
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

C:\Users\samue\anaconda3\envs\venv_py39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#importamos los paquetes necesarios para el cálculo de las métricas
import sklearn
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
import seaborn as sns

In [3]:
#en este script se entrenarán modelos que siguen una estructura idéntica a la relatada en los siguientes artículos:
# - Automatic Detection and Classification of Diabetic Retinopathy stages using CNN (Ghosh R.,Ghosh K.)
# - Classification of Diabetic Retinopathy Images Based on Customised CNN Architecture (Mobeen-ur-Rehman)
# - Diagnosis of retinal disorders from Optical Coherence Tomography images using CNN (Rajagopalan N., Venkateswaran N.)
# - AOCT-NET: a convolutional network automated classification of multiclass retinal diseases using spectral-domain optical coherence tomography images (Alqudah A.)

In [4]:
#establecemos el tamaño del batch, la escala de las imágenes y el número de épocas de entrenamiento
#debido a que cada una de las arquitecturas requiere una escala de imagen específica, la escala y los loaders serán definidas para cada arquitectura
batch = 4
#comenzaremos con la arquitectura propuesta por Ghosh, con una escala de 512, 512, 3
escala = 512
epocas = 50

In [5]:
#a continuación definimos la operación que permitirá transformar las imágenes del repositorio en Tensores que puedan ser empleados por PyTorch
transform = transforms.Compose(
    [transforms.ToTensor(), #transforma la imagen de formato PIL a formato tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #normaliza el tensor para que la media de sus valores sea 0 y su desviación estándar 0.5
     transforms.Resize((escala, escala))]) #redimensionamos las imágenes

In [6]:
#a continuación cargamos el conjunto de imágenes de train (OCT) y los dos de test (iPhone y Samsung)
OCT = ImageFolder(root = 'Datos/Classified Data/Images/OCT', transform = transform)
print(f'Tamaño del conjunto de datos de train: {len(OCT)}')

Samsung = ImageFolder(root = 'Datos/Classified Data/Images/Samsung', transform = transform)
print(f'Tamaño del conjunto de datos de test de Samsung: {len(Samsung)}')

iPhone = ImageFolder(root = 'Datos/Classified Data/Images/iPhone', transform = transform)
print(f'Tamaño del conjunto de datos de test de iPhone: {len(iPhone)}')

Tamaño del conjunto de datos de train: 113
Tamaño del conjunto de datos de test de Samsung: 93
Tamaño del conjunto de datos de test de iPhone: 99


In [7]:
#establecemos una lista con el nombre de las etiquetas
classes = OCT.classes

In [8]:
#y definimos también las funciones que van a ir cargando las imágenes en el modelo
train_loader = DataLoader(
    dataset = OCT,
    batch_size = 4, #establecemos un tamaño de lote (batch_size) de 4, ya que son pocas imágenes y podemos permitírnoslo
    shuffle = True, #indicamos que mezcle las imágenes
    num_workers = 2 #genera subprocesos para cargar los datos y así liberamos el proceso main
)

test_S_loader = DataLoader(
    dataset = Samsung,
    batch_size = 4, #establecemos un tamaño de lote (batch_size) de 10, ya que son pocas imágenes y podemos permitírnoslo
    shuffle = True, #indicamos que mezcle las imágenes
    num_workers = 2 #genera subprocesos para cargar los datos y así liberamos el proceso main
)

test_i_loader = DataLoader(
    dataset = iPhone,
    batch_size = 4, #establecemos un tamaño de lote (batch_size) de 10, ya que son pocas imágenes y podemos permitírnoslo
    shuffle = True, #indicamos que mezcle las imágenes
    num_workers = 2 #genera subprocesos para cargar los datos y así liberamos el proceso main
)

In [9]:
#en esta ocasión, para mayor sencillez del código, vamos a omitir el paso de representación de las imágenes (además sabemos que los DataLoaders funcionan correctamente pues son los mismos que los empleados en el script 'Primera_Red_Básica')

In [51]:
#ARQUITECTURA DEFINIDA POR GHOSH
class Ghosh(nn.Module):
    #esta estructura está formada por capas convolucionales, de maxpooling, de activación, de Dropout, fully-connected y de clasificación
    
    def __init__(self):
        #sobreescribimos el constructor del padre
        super(Ghosh,self).__init__()
        #primero definimos una capa convolucional
        self.conv1 = nn.Conv2d(
            in_channels = 3, #3 canales de entrada porque las imágenes son a color
            out_channels = 6, #se trata del número de salidas de la capa. Puede tratarse de un valor arbitrario
            kernel_size = 7, #suele tratarse de un número impar
            stride = 2, #cantidad píxeles que se desplaza el filtro sobre la imagen
            padding = 2, #cantidad de relleno que se va a aplicar sobre los bordes de la imagen
        )
        #una segunda capa convolucional
        self.conv2 = nn.Conv2d(
            in_channels = 6, #6 canales de entrada porque es el número de salidas de la capa anterior
            out_channels = 6, #en este caso deben coincidir entradas y salidas para que al llamar consecutivamente a dos capas convoucionales no haya problemas
            kernel_size = 3, #suele tratarse de un número impar
            stride = 1, #cantidad píxeles que se desplaza el filtro sobre la imagen
            padding = 1, #cantidad de relleno que se va a aplicar sobre los bordes de la imagen
        )
        #la función de activación (en este caso PReLU)
        self.activation = nn.PReLU()
        #la capa de MaxPool
        self.pool = nn.MaxPool2d(
            kernel_size = 2, #establecemos el tamaño del kernel a 2*2
            stride = 2 #cantidad píxeles que se desplaza el filtro sobre la imagen
        )
        #la primera capa de neuronas a la que aplicaremos Dropout como técnica de regularización
        self.fc1 = nn.Linear(
            in_features = 294, #número de parámetros de entrada de la red (los valores se obtienen experimentalmente)
            out_features = 256 #número de neuronas de salida
        )
        
        #la segunda capa fully-connected
        self.fc2 = nn.Linear(256,1024)
        
        #la tercera capa fully-connected
        self.fc3 = nn.Linear(1024,512)
        
        #la capa de neuronas fully-connected
        self.dense = nn.Linear(
            in_features = 512, #número de parámetros de entrada de la red (los valores se obtienen experimentalmente)
            out_features = 5 #número de neuronas de salida
        )
        #por último la capa de Softmax, que convierte los valores del Tensor predicho en probabilidades
        self.softmax = nn.Softmax(
            dim = 1 #dimensión sobre la que debe actuar softmax 
        )
        
    def forward(self,x):
        #en esta función es donde tiene lugar la computación (y la función invocada por defecto al ejecutar la red)
        x = self.pool(self.activation(self.conv1(x)))
        x = self.activation(self.conv2(x))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.activation(self.conv2(x))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.activation(self.conv2(x))
        x = self.activation(self.conv2(x))
        x = self.activation(self.conv2(x))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.activation(self.conv2(x))
        x = self.activation(self.conv2(x))
        x = self.activation(self.conv2(x))
        x = self.pool(self.activation(self.conv2(x)))
        x = x.view(-1,self.num_flat_features(x)) #usamos una función propia de la clase para obtener el número de características
        x = F.dropout(self.fc1(x))
        x = F.dropout(self.fc2(x))
        x = F.dropout(self.fc3(x))
        x = self.dense(x)
        x = self.softmax(x)
        
        return x
    
    def num_flat_features(self,x):
        #por último definimos la función que permite obtener el número de características de los tensores
        size = x.size()[1:] #seleccionamos todas las dimensiones expcepto la primera (que son los batches)
        num_features = 1
        for s in size:
            num_features*=s
        return num_features

In [52]:
ghosh = Ghosh()
print(ghosh)

Ghosh(
  (conv1): Conv2d(3, 6, kernel_size=(7, 7), stride=(2, 2), padding=(2, 2))
  (conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (activation): PReLU(num_parameters=1)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=294, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (dense): Linear(in_features=512, out_features=5, bias=True)
  (softmax): Softmax(dim=1)
)


In [53]:
#definimos como loss la función de tipo cross entropy 
criterion = nn.CrossEntropyLoss() 

In [54]:
#en este caso el optimizador será la función Adam (ampliamente utilizada)
optimizer = torch.optim.Adam(params = ghosh.parameters()) #dejamos el valor de learning rate por defecto (0.001)

In [55]:
#definimos 2 listas en las que almacenaremos los valores de accuracy y loss de cada época para poder graficarlo posteriormente
acc_graph = []
loss_graph = []
#para entrenar el modelo vamos a iterar el número de épocas determinadas, calculando el valor de loss y accuracy para cada época
for epoch in range(epocas):
    #establecemos el número de predicciones correctas inicial a 0
    correct = 0
    total = 0
    #y cargamos las imágenes de entrenamiento y sus etiquetas usando la estructura Loader previamente creada
    for i, data in enumerate(train_loader):
        inputs, labels = data
        #establecemos a 0 los parámetros del modelo
        optimizer.zero_grad()
        #generamos las predicciones de los inputs
        outputs = ghosh(inputs)
        #calculamos el loss, la desviación de las predicciones con respecto a las etiquetas
        loss = criterion(outputs, labels)
        #propagamos hacia atrás el valor loss
        loss.backward()
        #y modificamos los pesos en función del loss y la función optimizer
        optimizer.step()
        #actualizamos el número de predicciones correctas
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    #una vez finalizada la época (que recorre todo el conjunto de imágenes) mostramos el valor del loss y del accuracy
    print(f'Época {epoch +1}/{epocas} - Accuracy: {correct/len(OCT)} - Loss: {loss.data.item()}')
    #añadimos los valores a la lista correspondiente
    loss_graph.append(loss.data.item())
    acc_graph.append(correct/len(OCT))

Época 1/50 - Accuracy: 0.5132743362831859 - Loss: 1.9048324823379517
Época 2/50 - Accuracy: 0.5132743362831859 - Loss: 0.9048324823379517
Época 3/50 - Accuracy: 0.5132743362831859 - Loss: 1.9048324823379517
Época 4/50 - Accuracy: 0.5132743362831859 - Loss: 1.9048324823379517
Época 5/50 - Accuracy: 0.5132743362831859 - Loss: 0.9048324823379517
Época 6/50 - Accuracy: 0.5132743362831859 - Loss: 0.9048324823379517


KeyboardInterrupt: 